In [1]:
import numpy as np 
import torch, gc
import torchvision.models as models
import torchvision.datasets as dsets
import torch.nn as nn
from torch.utils.data import DataLoader
from typing import Dict, Iterable, Callable


from tqdm import tqdm 

torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = False
torch.backends.cudnn.enabled = True
torch.backends.cudnn.allow_tf32 = False
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# gc.collect()
# torch.cuda.empty_cache()

# model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).to('cuda')
# model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1).to('cuda')
# model = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1).to('cuda')
model = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1).to('cuda')
def Quant(x : torch.Tensor, n : int) :

    N = 2 ** n
    N_MIN, N_MAX = -N//2, N//2 - 1
    x_max, x_min = torch.max(x) , torch.min(x)

    scale = (x_max - x_min) / (N-1)
    scale += (x_max * (scale == 0))
    zero_n = x_max * N_MIN - x_min * N_MAX
    zero_d = x_max - x_min
    zero_p =  torch.round(zero_n / (zero_d + 1e-30)) * (zero_d != 0)

    x_hat = torch.round(x / scale + zero_p)
    x_q   = torch.clip(x_hat, N_MIN, N_MAX).type(torch.int16)

    return x_q, scale, zero_p
     
def DeQuant(    x_q: torch.Tensor, 
                scale: torch.Tensor, 
                zero_p: torch.Tensor):
    return scale  * (x_q - zero_p)

def save_outputs_hook(self, layer_id = str) -> Callable:          
    def fn(_, input) :
        with torch.no_grad():
            Quant_input, scale, zero_p = Quant(input[0],16)
            # print (Quant_input.type)
            input[0][:] = DeQuant(Quant_input, scale, zero_p)
    return fn

for name, layer in model.named_modules():
    if ("layer1" != name) | ("layer2" != name) | ("layer3" != name)| ("layer4" != name) :
        layer = dict([*model.named_modules()])[name]
        layer.register_forward_pre_hook(save_outputs_hook(name))

for name, param in model.named_parameters():
    Data_shape = param.shape
    with torch.no_grad():
        print("origin : ",param.view(torch.int16).view(-1))
        Quant_input, scale, zero_p = Quant(param,16)
        param[:] = DeQuant(Quant_input, scale, zero_p)    
    Data_1d = param.view(-1)
    print("Converted : ",Data_1d.view(torch.int16))
     
dataset = dsets.ImageFolder("/media/imagenet/val", models.EfficientNet_B2_Weights.IMAGENET1K_V1.transforms()) ### 2번째 인자, transform
loader = DataLoader(dataset= dataset, # dataset
                   batch_size=128,   # batch size power to 2
                   shuffle = False, # false
                   num_workers = 8, # num_workers 
                   pin_memory=True) # pin_memory 

correct = 0
total = 50000

model.eval()
# torch.no_grad()
with torch.no_grad():
    for input, label in tqdm(loader):
        input = input.cuda(non_blocking=True)
        label = label.cuda(non_blocking=True)     
        output = model(input)    
        pred = torch.argmax(output, 1)
        correct += (pred == label).int().sum()
    acc1 = correct / total * 100

print(acc1)

/home/kkh/anaconda3/envs/1031_kkh/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


origin :  tensor([12146, 15674, 15530,  ..., 15875,  8370, 16205], device='cuda:0',
       dtype=torch.int16)
Converted :  tensor([ 9384, 15674, 15253,  ..., 15875,  7988, 16205], device='cuda:0',
       dtype=torch.int16)
origin :  tensor([-15454,  16409,  23411,  16361, -13765,  16487,   7355,  16231,  -1550,
         16460,  -5160,  16173,  10866,  16076,   7586,  16513,   6256,  16115,
         20219,  16378, -31170,  16537, -16475,  16400,  27306,  16323,   2131,
        -16362,  14157,  16557,  18312,  16398, -28278,  16034, -11522,  16134,
        -10505,  16164,   -122, -16304, -19273,  16556,   9011,  16296,  -4400,
         16289, -29901,  16575, -10797,  16452,  21288,  16361, -14472,  16160,
          3678,  16487,  17704,  16409, -11486,  16050,  -9950,  16432,   -648,
         16366], device='cuda:0', dtype=torch.int16)
Converted :  tensor([-15685,  16409,  23917,  16361, -13571,  16487,   6966,  16231,  -1436,
         16460,  -4257,  16173,  11995,  16076,   7481,  1651

100%|██████████| 391/391 [06:31<00:00,  1.00s/it]

tensor(80.6000, device='cuda:0')


In [2]:
import numpy as np
import math
import time
import sys

np.set_printoptions(threshold=np.inf, linewidth=np.inf)
#test = np.load('/users/user2/Desktop/AIDC/model/features.npy').reshape(-1, 64)
# print(test.shape)

# data = test[383]
# data = data.astype(np.uint16)
# print(len(data))
# print(data)
N = 64
M = 16
datatype = 'uint16'

# decimal int number to binary numpy array
def toBinary(value, wid):
    value_bin = np.binary_repr(value, width = wid)
    # width가 자릿수, value가 바꿀 정수
    
    value_bin = np.array(list(value_bin))
    value_bin = value_bin.astype(datatype)
    return value_bin

# a = 2
# print(toBinary(a, 7))

# binary numpy array to decimal int number
def toDecimal(nparr):
    fliarr = np.flip(nparr)
    # 들어온 넘피 어레이의 끝자리부터 2곱해서 더해줄려고 1차원 넘피 어레이를 뒤집어주는 함수
    # ex) [0 0 0 0 0 1] => [1 0 0 0 0 0]
    
    twoarr = np.array([2**0,2**1,2**2,2**3,2**4,2**5,2**6,2**7,2**8,2**9,2**10,2**11,2**12,2**13,2**14,2**15], dtype = np.uint16)
    valuearr = fliarr * twoarr[0:len(fliarr)]
    value = valuearr.sum()
    return value

# b = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1], dtype = datatype)
# print(toDecimal(b))
# print(b.sum())

def Delta(datablock):
    baseword = datablock[0]
    delta = datablock[1:].astype(np.uint16) - baseword    
    delta = delta.astype(datatype)
    
    return baseword, delta

# baseword, delta= Delta(data)
# print(baseword)
# print(delta)
# print(len(delta))

# change to bit plane
def DBP16(deltablock):
    deltas = np.unpackbits(deltablock.view(np.uint8)).reshape(-1, 16)
    d1 = deltas[:,0:8]
    d2 = deltas[:,8:16]
    deltas = np.hstack((d2, d1))
    deltas = deltas.astype(datatype)
    
    
    dbps = np.array(list(zip(*deltas[::]))) ## 2차원 넘피 어레이를 90도 회전시키는 함수
    return dbps

# dbps16 = DBP16(delta)
# print(dbps16)
# print(dbps16.shape)

# calculate to delta-bit plane-xor
def DBX16(dbps):
    x1 = np.delete(dbps, 0, 0).astype(np.uint16)
    x2 = np.delete(dbps, dbps.shape[0] - 1, 0).astype(np.uint16)
    xored = x1 ^ x2
    xored = np.vstack([dbps[0], xored])
    return xored

# dbxs16 = DBX16(dbps16)
# print(dbxs16)
# print(dbxs16.shape)

def Encoder(dbp, dbx):
    
    global patt0
    
    # if DBX plane is all 1 or all 0
    if np.all(dbx==0):
        return 3, np.array([0, 0, 1], dtype = datatype)
    elif np.all(dbx==1):
        return 5, np.array([0, 0, 0, 0, 0], dtype = datatype)
    elif np.all(dbp==0):
        return 5, np.array([0, 0, 0, 0, 1], dtype = datatype)
    
    
    # plane is not all 1 or all 0
    pos_one = np.where(dbx == 1)[0]
    
    # single 1
    if len(pos_one) == 1:
        return 5 + 6, np.concatenate((np.array([0, 0, 0, 1, 1], dtype = datatype), toBinary(62 - pos_one[0], 6))) ## len(delta)가 이미 N-1이므로, 바로 log2를 취하면 됩니다
    
    
    elif len(pos_one) == 2:
        # Consecutive two 1 연속된 1
        if pos_one[1] - pos_one[0] == 1:
            return 5 + 6, np.concatenate((np.array([0, 0, 0, 1, 0], dtype = datatype), toBinary(62 - pos_one[1], 6))) ## 똑같이 -2가 아니라 -1만 하면 됨
        
        # 연속되지 않은 1 => not compress
        else:
            return len(delta)+1, np.concatenate((np.array([1], dtype=datatype), dbx))
        
    else:
        return len(delta)+1, np.concatenate((np.array([1], dtype=datatype), dbx))
   
# for i in range(M):
#     length, code = Encoder(dbps16[i], dbxs16[i])
#     print(dbxs16[i], "->", length, code)

def BPC(block):
    outputcode = np.array([], dtype=datatype)
    outputlen = 0
    
    baseword, deltablock = Delta(block)
    dbps = DBP16(deltablock)
    dbxs = DBX16(dbps)
    basesymbol = toBinary(baseword, 16)

    zrl = 0

    for i in range(M):
        length, code = Encoder(dbps[i], dbxs[i])

        if np.array_equal(code, np.array([0, 0, 1], dtype = datatype)): # all-0 DBX 일 경우
            zrl += 1
            continue

        else: # all-0 DBX가 아닐 경우
            if zrl != 0: # 근데 앞에 all-0 DBX가 있었을 경우
                if zrl == 1:
                    outputcode = np.concatenate((outputcode, np.array([0, 0, 1], dtype = datatype)), axis=None)
                    outputlen += 3
                else:
                    runlen = toBinary(zrl-2, 4) 
                    outputcode = np.concatenate((outputcode, np.array([0, 1], dtype=datatype), runlen), axis=None)
                    outputlen += 2 + 4
                # 이제 all-0 DBX가 아닌 현재 code를 붙여줘야함
                outputcode = np.concatenate((outputcode, code), axis=None)
                outputlen += length
                zrl = 0
            else: # 앞에 all-0 DBX가 없었을 경우
                outputcode = np.concatenate((outputcode, code), axis=None)
                outputlen += length
    
    if zrl != 0:
        if zrl == 1:
            outputcode = np.concatenate((outputcode, np.array([0, 0, 1], dtype = datatype)), axis=None)
            outputlen += 3
        else:
            runlen = toBinary(zrl-2, 4) 
            outputcode = np.concatenate((outputcode, np.array([0, 1], dtype=datatype), runlen), axis=None)
            outputlen += 2 + 4
        
    outputlen += 16
    outputcode = np.concatenate((basesymbol, outputcode), axis=None)
    
    outputlen += 2
    outputcode = np.concatenate((np.array([0, 0], dtype=datatype), outputcode), axis=None)
    
    
    # padding
    if outputlen < 512:
        e = 512 - outputlen
        ex = np.zeros(e, dtype=datatype)
        outputcode = np.concatenate((outputcode, ex), axis=None)

    
    return outputcode, outputlen

# result_bpc, len_bpc = BPC(data)
# print(result_bpc)
# print(len_bpc)
# print(len(result_bpc))

def ZRLE(block):
    outputcode = np.array([], dtype=datatype)
    outputlen = 0
    new_block = block.reshape(-1, 4)
    for idx in range(new_block.shape[0]):
        vec = np.where(new_block[idx] > 0)[0]
        if len(vec) == 0:
            outputcode = np.concatenate((outputcode, np.array([0, 0, 0, 0, 0, 0], dtype=datatype)), axis=None)
            outputlen += 6
        elif len(vec) == 1:
            if vec[0] == 3:
                outputcode = np.concatenate((outputcode, np.array([0, 0, 0, 0, 0, 1], dtype=datatype)), axis=None)
                outputlen += 6
            elif vec[0] == 2:
                outputcode = np.concatenate((outputcode, np.array([0, 0, 0, 0, 1], dtype=datatype)), axis=None)
                outputlen += 5
            elif vec[0] == 1:
                outputcode = np.concatenate((outputcode, np.array([0, 0, 0, 1, 0], dtype=datatype)), axis=None)
                outputlen += 5
            elif vec[0] == 0:
                outputcode = np.concatenate((outputcode, np.array([0, 0, 0, 1, 1], dtype=datatype)), axis=None)
                outputlen += 5
            outputcode = np.concatenate((outputcode, toBinary(new_block[idx][vec[0]], 16)), axis=None)
            outputlen += 16
        elif len(vec) == 2:
            if vec[1] == 3:
                if vec[0] == 2:
                    outputcode = np.concatenate((outputcode, np.array([0, 0, 1, 0], dtype=datatype)), axis=None)
                    outputlen += 4
                elif vec[0] == 1:
                    outputcode = np.concatenate((outputcode, np.array([0, 0, 1, 1], dtype=datatype)), axis=None)
                    outputlen += 4
                elif vec[0] == 0:
                    outputcode = np.concatenate((outputcode, np.array([0, 1, 0, 0], dtype=datatype)), axis=None)
                    outputlen += 4
            elif vec[1] == 2:
                if vec[0] == 1:
                    outputcode = np.concatenate((outputcode, np.array([0, 1, 0, 1], dtype=datatype)), axis=None)
                    outputlen += 4
                elif vec[0] == 0:
                    outputcode = np.concatenate((outputcode, np.array([0, 1, 1, 0], dtype=datatype)), axis=None)
                    outputlen += 4
            elif vec[1] == 1:
                outputcode = np.concatenate((outputcode, np.array([0, 1, 1, 1], dtype=datatype)), axis=None)
                outputlen += 4
            outputcode = np.concatenate((outputcode, toBinary(new_block[idx][vec[0]], 16), toBinary(new_block[idx][vec[1]], 16)), axis=None)
            outputlen += 32
        elif len(vec) == 3:
            sumvec = vec[0] + vec[1] + vec[2]
            if sumvec == 6: #123
                outputcode = np.concatenate((outputcode, np.array([1, 0, 0, 0], dtype=datatype)), axis=None)
                outputlen += 4
            elif sumvec == 5: #023
                outputcode = np.concatenate((outputcode, np.array([1, 0, 0, 1], dtype=datatype)), axis=None)
                outputlen += 4
            elif sumvec == 4: #013
                outputcode = np.concatenate((outputcode, np.array([1, 0, 1, 0], dtype=datatype)), axis=None)
                outputlen += 4
            elif sumvec == 3: #012
                outputcode = np.concatenate((outputcode, np.array([1, 0, 1, 1], dtype=datatype)), axis=None)
                outputlen += 4
            outputcode = np.concatenate((outputcode, toBinary(new_block[idx][vec[0]], 16), toBinary(new_block[idx][vec[1]], 16), toBinary(new_block[idx][vec[2]], 16)), axis=None)
            outputlen += 48
        elif len(vec) == 4:
            outputcode = np.concatenate((outputcode, np.array([1, 1], dtype=datatype)), axis=None)
            outputcode = np.concatenate((outputcode, toBinary(new_block[idx][vec[0]], 16), toBinary(new_block[idx][vec[1]], 16), toBinary(new_block[idx][vec[2]], 16), toBinary(new_block[idx][vec[3]], 16)), axis=None)
            outputlen += 66

    outputcode = np.concatenate((np.array([0, 1], dtype=datatype), outputcode), axis=None)
    outputlen += 2
    
    # padding
    if outputlen < 512:
        e = 512 - outputlen
        ex = np.zeros(e, dtype=datatype)
        outputcode = np.concatenate((outputcode, ex), axis=None)
    
    return outputcode, outputlen

# result_zrl, len_zrl = ZRLE(data)
# print(result_zrl)
# print(len_zrl)
# print(len(result_zrl))

def SR(block):
    reduct_block = np.logical_or(block < 128, block > 65407)
    idx_block = np.where(block > 65407)[0]
    dup_block = block.copy()
    outputcode = np.array([], dtype=datatype)
    if np.all(reduct_block == True):
        if (block[0] < 64 or block[0] > 65471):
            ret_flag = 1
        else:
            ret_flag = 0
    else:
        ret_flag = 0
    
    if (dup_block[0] > 65407):
        dup_block[0] -= 128
    dup_block = dup_block.astype(np.uint8)
    
    for idx in range(len(dup_block)):
        if idx == 0:
            outputcode = np.concatenate((outputcode, np.array([1], dtype=datatype), toBinary(dup_block[idx], 7)[-7:]), axis=None)
        else:
            outputcode = np.concatenate((outputcode, toBinary(dup_block[idx], 8)), axis=None)
    
    return outputcode, ret_flag

# result_sr, len_sr = SR(data)
# print(result_sr)
# print(len_sr)
# print(len(result_sr))

# if len_sr == 1:
#     print(result_sr)
# elif len_zrl < 513:
#     print(result_zrl)
# elif len_bpc < 513:
#     print(result_bpc)
# else:
#     print(result_bpc[:512])
import torch

def Comp(x : torch.Tensor):  
     x_numpy = x.numpy()
     newarr = np.array([], dtype=np.uint16)
     npline = np.array(list(x_numpy[:-1]), dtype=np.uint16).reshape(-1, 16)
     print(f"npline : {npline.shape}")
     
     for i in range(64):
          newarr = np.append(newarr, toDecimal(npline[i]), axis=None)
     bpc_code, bpc_len = BPC(newarr)
     zrl_code, zrl_len = ZRLE(newarr)
     sr_code, sr_flag = SR(newarr)
     
     # print(f"sr_flag : {sr_flag} sr_code : {sr_code}")
     # print(f"bpc_len : {bpc_len} bpc_code : {bpc_code}")
     # print(f"zrl_len : {zrl_len} zrl_code : {zrl_code}")
     if sr_flag == 1:
          result = sr_code
     elif zrl_len < 513:
          result = zrl_code
     elif bpc_len < 513:
          result = bpc_code
     else:
          result = bpc_code[0:512]
     return result


x = torch.rand(10,3,224,224)
Comp(x)

npline : (84672, 16)


array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
import numpy as np
import math
import time
import sys

def sr_decomp(data_i):
    data_o = np.array((), dtype="int")
    if data_i[1] == 1:
        code0 = np.array([1,1,1,1,1,1,1,1,1], dtype="int")
    else:
        code0 = np.array([0,0,0,0,0,0,0,0,0], dtype="int")
    code0 = np.concatenate((code0, data_i[1:8]), dtype="int")
    data_o = np.concatenate((data_o, code0), dtype="int", axis=0)
    for i in range(1,64):
        if data_i[i*8] == 1:
            code_i = np.array([1,1,1,1,1,1,1,1], dtype="int")
        else:
            code_i = np.array([0,0,0,0,0,0,0,0], dtype="int")
        code_i = np.concatenate((code_i, data_i[i*8:(i+1)*8]), dtype="int")
        data_o = np.concatenate((data_o, code_i), dtype="int")
    return data_o

def zrl_decomp(data_i):
    data_o = np.array((), dtype="int")
    cnt = 0
    idx = 2
    while cnt < 16:
        if np.array_equal(data_i[idx:idx+2], np.array([1,1], dtype="int")):
            code_i=data_i[idx+2:idx+66]
            cnt=cnt+1
            idx=idx+66
        elif np.array_equal(data_i[idx:idx+4], np.array([1,0,1,1], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[0:16]=data_i[idx+4:idx+20]
            code_i[16:32]=data_i[idx+20:idx+36]
            code_i[32:48]=data_i[idx+36:idx+52]
            cnt=cnt+1
            idx=idx+52
        elif np.array_equal(data_i[idx:idx+4], np.array([1,0,1,0], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[0:16]=data_i[idx+4:idx+20]
            code_i[16:32]=data_i[idx+20:idx+36]
            code_i[48:64]=data_i[idx+36:idx+52]
            cnt=cnt+1
            idx=idx+52
        elif np.array_equal(data_i[idx:idx+4], np.array([1,0,0,1], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[0:16]=data_i[idx+4:idx+20]
            code_i[32:48]=data_i[idx+20:idx+36]
            code_i[48:64]=data_i[idx+36:idx+52]
            cnt=cnt+1
            idx=idx+52
        elif np.array_equal(data_i[idx:idx+4], np.array([1,0,0,0], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[16:32]=data_i[idx+4:idx+20]
            code_i[32:48]=data_i[idx+20:idx+36]
            code_i[48:64]=data_i[idx+36:idx+52]
            cnt=cnt+1
            idx=idx+52
        elif np.array_equal(data_i[idx:idx+4], np.array([0,1,1,1], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[0:16]=data_i[idx+4:idx+20]
            code_i[16:32]=data_i[idx+20:idx+36]
            cnt=cnt+1
            idx=idx+36
        elif np.array_equal(data_i[idx:idx+4], np.array([0,1,1,0], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[0:16]=data_i[idx+4:idx+20]
            code_i[32:48]=data_i[idx+20:idx+36]
            cnt=cnt+1
            idx=idx+36
        elif np.array_equal(data_i[idx:idx+4], np.array([0,1,0,1], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[16:32]=data_i[idx+4:idx+20]
            code_i[32:48]=data_i[idx+20:idx+36]
            cnt=cnt+1
            idx=idx+36
        elif np.array_equal(data_i[idx:idx+4], np.array([0,1,0,0], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[0:16]=data_i[idx+4:idx+20]
            code_i[48:64]=data_i[idx+20:idx+36]
            cnt=cnt+1
            idx=idx+36
        elif np.array_equal(data_i[idx:idx+4], np.array([0,0,1,1], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[16:32]=data_i[idx+4:idx+20]
            code_i[48:64]=data_i[idx+20:idx+36]
            cnt=cnt+1
            idx=idx+36
        elif np.array_equal(data_i[idx:idx+4], np.array([0,0,1,0], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[32:48]=data_i[idx+4:idx+20]
            code_i[48:64]=data_i[idx+20:idx+36]
            cnt=cnt+1
            idx=idx+36
        elif np.array_equal(data_i[idx:idx+5], np.array([0,0,0,1,1], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[0:16]=data_i[idx+5:idx+21]
            cnt=cnt+1
            idx=idx+21
        elif np.array_equal(data_i[idx:idx+5], np.array([0,0,0,1,0], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[16:32]=data_i[idx+5:idx+21]
            cnt=cnt+1
            idx=idx+21
        elif np.array_equal(data_i[idx:idx+5], np.array([0,0,0,0,1], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[32:48]=data_i[idx+5:idx+21]
            cnt=cnt+1
            idx=idx+21
        elif np.array_equal(data_i[idx:idx+6], np.array([0,0,0,0,0,1], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            code_i[48:64]=data_i[idx+6:idx+22]
            cnt=cnt+1
            idx=idx+22
        elif np.array_equal(data_i[idx:idx+6], np.array([0,0,0,0,0,0], dtype="int")):
            code_i=np.zeros(64, dtype="int")
            cnt=cnt+1
            idx=idx+6
        data_o=np.concatenate((data_o, code_i), dtype="int")

    return data_o

def bpc_decomp(data_i):
    data_o = np.array((), dtype="int")
    idx = 18;
    base = data_i[2:18]
    dbps = Decoder(data_i[18:512]) # make dbps with 494bit data
    
    deltas = Bitplane(dbps)
    
    origin = Adder(base, deltas)
    
    for i in range(16):
        code_o = origin[i]
        data_o = np.concatenate((data_o,code_o), dtype="int")
    return data_o

def toDecimal(nparr):
    fliarr = np.flip(nparr)
    twoarr = np.array([2**0,2**1,2**2,2**3,2**4,2**5,2**6,2**7,2**8,2**9,2**10,2**11,2**12,2**13,2**14,2**15], dtype="int")
    valuearr = fliarr * twoarr[0:len(fliarr)]
    value = valuearr.sum()
    return value

def toBinary(value, wid):
    value_bin = np.binary_repr(value, width=wid)
    if len(value_bin) > wid:
        sub = len(value_bin) - wid
        value_bin = value_bin[sub:sub+16]
    value_bin = np.array(list(value_bin), dtype="int")
    return value_bin

def Decoder(data_i):
    dbps_d = np.zeros((16,63), dtype="int")
    idx_d = 0
    zrl_cnt = 0
    do_xor = 0
    cnt_d = 0
    
    while (cnt_d < 16) & (idx_d < 494):
        if zrl_cnt > 0:
            code = np.zeros(63, dtype="int")
            do_xor=1
            zrl_cnt = zrl_cnt-1;
        else :
            if data_i[idx_d]:
                if idx_d < 430:
                    code=data_i[idx_d+1:idx_d+64]
                    do_xor=1
                    idx_d=idx_d+64
                else :
                    code=np.zeros(63, dtype="int")
                    do_xor=0
                    idx_d=494
            elif np.array_equal(data_i[idx_d:idx_d+2], np.array([0,1], dtype="int")):
                if idx_d < 488:
                    code = np.zeros(63, dtype="int")
                    do_xor=1
                    zrl_cnt = toDecimal(data_i[idx_d+2:idx_d+6])+1
                    idx_d=idx_d+6
                else :
                    code = np.zeros(63, dtype="int")
                    do_xor=0
                    idx_d=494
            elif np.array_equal(data_i[idx_d:idx_d+3], np.array([0,0,1], dtype="int")):
                if idx_d < 489:
                    code = np.zeros(63, dtype="int")
                    do_xor=1
                    idx_d=idx_d+3
                else :
                    code = np.zeros(63, dtype="int")
                    do_xor=0
                    idx_d=494
            elif np.array_equal(data_i[idx_d:idx_d+5], np.array([0,0,0,0,0], dtype="int")):
                if idx_d < 489:
                    code = np.ones(63, dtype="int")
                    do_xor=1
                    idx_d=idx_d+5
                else :
                    code = np.zeros(63, dtype="int")
                    do_xor=0
                    idx_d=494
            elif np.array_equal(data_i[idx_d:idx_d+5], np.array([0,0,0,0,1], dtype="int")): # do not xor
                if idx_d < 489:
                    code = np.zeros(63, dtype="int")
                    do_xor=0
                    idx_d=idx_d+5
                else :
                    code = np.zeros(63, dtype="int")
                    do_xor=0
                    idx_d=494
            elif np.array_equal(data_i[idx_d:idx_d+5], np.array([0,0,0,1,0], dtype="int")):
                if idx_d < 483:
                    code = np.zeros(63, dtype="int")
                    bin_pos=data_i[idx_d+5:idx_d+11]
                    pos=toDecimal(bin_pos)
                    code[pos]=1
                    code[pos+1]=1
                    do_xor=1
                    idx_d=idx_d+11
                else :
                    code = np.zeros(63, dtype="int")
                    do_xor=0
                    idx_d=494
            elif np.array_equal(data_i[idx_d:idx_d+5], np.array([0,0,0,1,1], dtype="int")):
                if idx_d < 483:
                    code = np.zeros(63, dtype="int")
                    bin_pos=data_i[idx_d+5:idx_d+11]
                    pos=toDecimal(bin_pos)
                    code[pos]=1
                    do_xor=1
                    idx_d=idx_d+11
                else :
                    code = np.zeros(63, dtype="int")
                    do_xor=0
                    idx_d=494
        
        if cnt_d == 0:
            dbps_d[cnt_d] = code
        else:
            if do_xor:
                dbps_d[cnt_d] = code ^ dbps_d[cnt_d-1]
            else:
                dbps_d[cnt_d] = code
        do_xor=0
        
        cnt_d=cnt_d+1
    return dbps_d

def Bitplane(dbps):
    deltas = np.array(list(zip(*dbps[::])))
    return deltas

def adderBintoBin(num1, num2):
    deci_num1 = toDecimal(num1)
    deci_num2 = toDecimal(num2)
    out_num = deci_num1 + deci_num2
    bin_out = toBinary(out_num, 16)
    return bin_out

def Adder(base, deltas):
    origin = np.zeros((16,64), dtype="int")
    idx=0
    while idx < 16:
        if idx == 0:
            data_1000 = base
        else :
            data_1000 = adderBintoBin(base, deltas[4*idx-1])
        data_0100 = adderBintoBin(base, deltas[4*idx])
        data_0010 = adderBintoBin(base, deltas[4*idx+1])
        data_0001 = adderBintoBin(base, deltas[4*idx+2])
        
        origin[idx] = np.concatenate((data_1000, data_0100, data_0010, data_0001), dtype="int")
        idx=idx+1
    return origin

def DECOMP_TOP(data_i):
    if data_i[0:2] == '00': mode = 0
    elif data_i[0:2] == '01': mode = 1
    elif data_i[0] == '1': mode = 2
    else : mode = 3 # error
        
    ndata_i = np.array(list(data_i[:-1]), dtype="int16") # without \n
    
    if mode == 0:
        data_o = bpc_decomp(ndata_i)
    elif mode == 1:
        data_o = zrl_decomp(ndata_i)
    elif mode == 2:
        data_o = sr_decomp(ndata_i)
    else :
        data_o = np.array([], dtype="int16")
    str_data_o = "".join(data_o.astype('str'))
    return data_o, mode

def DECOMP(Comped : torch.Tensor, origin : torch.Tensor):
    bcnt=1
    su_cnt=0
    fl_cnt=0
    zrlcnt=0
    bpccnt=0
    srcnt=0

    while True:
        data_i = read_f.readline()
        cdata_i = compr_f.readline()
        if data_i == "": break;
            
        ndata_i = data_i
        ncdata_i = cdata_i[:-1]
        
        data_o, mode = DECOMP_TOP(data_i)

        if ncdata_i == data_o : 
            su_cnt=su_cnt+1
        else :
            fl_cnt=fl_cnt+1
            
        if mode == 0:
            bpccnt=bpccnt+1
        elif mode == 1:
            zrlcnt=zrlcnt+1
        elif mode == 2:
            srcnt=srcnt+1
            
        bcnt=bcnt+1
        write_f.write(data_o)
        write_f.write("\n")

    print(f'compression with bpc = {bpccnt}')
    print(f'compression with zrl = {zrlcnt}')
    print(f'compression with sr  = {srcnt}')
    print(f'success count = {su_cnt}')
    print(f'success rate = {su_cnt/bcnt*100}%')
    print(f'faliure count = {fl_cnt}')
    print(f'faliure rate = {fl_cnt/bcnt*100}%')


In [ ]:
import torch
def Quant(x : torch.Tensor, n : int) :

    N = 2 ** n
    N_MIN, N_MAX = -N//2, N//2 - 1
    x_max, x_min = torch.max(x) , torch.min(x)

    scale = (x_max - x_min) / (N-1)
    scale += (x_max * (scale == 0))
    zero_n = x_max * N_MIN - x_min * N_MAX
    zero_d = x_max - x_min
    zero_p =  torch.round(zero_n / (zero_d + 1e-30)) * (zero_d != 0)

    x_hat = torch.round(x / scale + zero_p)
    x_q   = torch.clip(x_hat, N_MIN, N_MAX).type(torch.int16)

    return x_q, scale, zero_p
     
     
     
def DeQuant(   x_q: torch.Tensor, 
                        scale: torch.Tensor, 
                        zero_p: torch.Tensor):
    return scale  * (x_q - zero_p)


x = torch.rand(10,3,224,224)
Quant(x,16)
    